In [2]:
import requests
from bs4 import BeautifulSoup

In [17]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Mostly clear, with a low around 47. West northwest wind 9 to 14 mph becoming light northwest  after midnight. Winds could gust as high as 24 mph. " class="forecast-icon" src="newimages/medium/nfew.png" title="Tonight: Mostly clear, with a low around 47. West northwest wind 9 to 14 mph becoming light northwest  after midnight. Winds could gust as high as 24 mph. "/>
 </p>
 <p class="short-desc">
  Mostly Clear
 </p>
 <p class="temp temp-low">
  Low: 47 °F
 </p>
</div>


In [21]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Tonight
Mostly Clear
Low: 47 °F


In [22]:
img = tonight.find("img")
desc = img['title']
print(desc)

Tonight: Mostly clear, with a low around 47. West northwest wind 9 to 14 mph becoming light northwest  after midnight. Winds could gust as high as 24 mph. 


In [24]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Tonight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight']

In [26]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Mostly Clear', 'Sunny andBreezy thenSunny', 'Clear', 'Sunny', 'Mostly Clear', 'Mostly Sunny', 'Slight ChanceRain', 'Partly Sunny', 'Mostly Cloudy']
['Low: 47 °F', 'High: 66 °F', 'Low: 45 °F', 'High: 66 °F', 'Low: 44 °F', 'High: 62 °F', 'Low: 48 °F', 'High: 64 °F', 'Low: 50 °F']
['Tonight: Mostly clear, with a low around 47. West northwest wind 9 to 14 mph becoming light northwest  after midnight. Winds could gust as high as 24 mph. ', 'Thursday: Sunny, with a high near 66. Breezy, with a north wind 17 to 23 mph, with gusts as high as 33 mph. ', 'Thursday Night: Clear, with a low around 45. North wind 7 to 13 mph, with gusts as high as 18 mph. ', 'Friday: Sunny, with a high near 66. Calm wind becoming west northwest around 6 mph in the afternoon. ', 'Friday Night: Mostly clear, with a low around 44. West wind 6 to 14 mph. ', 'Saturday: Mostly sunny, with a high near 62.', 'Saturday Night: A 20 percent chance of rain after 11pm.  Mostly cloudy, with a low around 48.', 'Sunday: Partly s

In [27]:
import pandas as pd 
weather = pd.DataFrame({
    "period" : periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc": descs
})
weather

,period,short_desc,temp,desc
0,Tonight,Mostly Clear,Low: 47 °F,"Tonight: Mostly clear, with a low around 47. W..."
1,Thursday,Sunny andBreezy thenSunny,High: 66 °F,"Thursday: Sunny, with a high near 66. Breezy, ..."
2,ThursdayNight,Clear,Low: 45 °F,"Thursday Night: Clear, with a low around 45. N..."
3,Friday,Sunny,High: 66 °F,"Friday: Sunny, with a high near 66. Calm wind ..."
4,FridayNight,Mostly Clear,Low: 44 °F,"Friday Night: Mostly clear, with a low around ..."
5,Saturday,Mostly Sunny,High: 62 °F,"Saturday: Mostly sunny, with a high near 62."
6,SaturdayNight,Slight ChanceRain,Low: 48 °F,Saturday Night: A 20 percent chance of rain af...
7,Sunday,Partly Sunny,High: 64 °F,"Sunday: Partly sunny, with a high near 64."
8,SundayNight,Mostly Cloudy,Low: 50 °F,"Sunday Night: Mostly cloudy, with a low around..."


In [31]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    47
1    66
2    45
3    66
4    44
5    62
6    48
7    64
8    50
Name: temp_num, dtype: object

In [32]:
weather["temp_num"].mean()

54.666666666666664

In [33]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool

In [34]:
weather[is_night]

,period,short_desc,temp,desc,temp_num,is_night
0,Tonight,Mostly Clear,Low: 47 °F,"Tonight: Mostly clear, with a low around 47. W...",47,True
2,ThursdayNight,Clear,Low: 45 °F,"Thursday Night: Clear, with a low around 45. N...",45,True
4,FridayNight,Mostly Clear,Low: 44 °F,"Friday Night: Mostly clear, with a low around ...",44,True
6,SaturdayNight,Slight ChanceRain,Low: 48 °F,Saturday Night: A 20 percent chance of rain af...,48,True
8,SundayNight,Mostly Cloudy,Low: 50 °F,"Sunday Night: Mostly cloudy, with a low around...",50,True
